In [1]:
import pandas as pd
import numpy as np
import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.geometry import Point
import geoplot as gplt

## Import The Datasets to Pandas

In [2]:
Uber2015=pd.read_csv('uber_trips_2015.csv')
Uber2014=pd.read_csv('uber_trips_2014.csv')
Demographics=pd.read_csv('demographics.csv')
Geographics=pd.read_csv('geographic.csv')
Green=pd.read_csv('green_trips_new_2.csv')
Subway=pd.read_csv('mta_trips.csv')
Weather=pd.read_csv('weather.csv') 
Yellow=pd.read_csv('yellow_trips_new.csv')
Zones=pd.read_csv('zones.csv')

C:\Users\Santiago\Anaconda3\envs\DSA4_TEST\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# List of column names for the Transport clean table

In [3]:
TransportCols =['TransportType','PickDateTime','DropDateTime','PickNTA','DropNTA','Borough','Year','PickHour','DropHour',
                'Distance','DeltaTime','Cost','MaxTemp','MinTemp','AvgTemp','Precipitation','Snowfall','SnowDepth','WeekDay']
TransportCols

['TransportType',
 'PickDateTime',
 'DropDateTime',
 'PickNTA',
 'DropNTA',
 'Borough',
 'Year',
 'PickHour',
 'DropHour',
 'Distance',
 'DeltaTime',
 'Cost',
 'MaxTemp',
 'MinTemp',
 'AvgTemp',
 'Precipitation',
 'Snowfall',
 'SnowDepth',
 'WeekDay']

# Coordinates to NTA

In [4]:
NTA=Geographics.copy()

## Make list of NTA columns

Boro=NTA.columns

# Create the GeoDataframe Object of each NTA polygon

NTAmap = gpd.GeoDataFrame()
c=0
for i in Boro:
    temp=NTA[i].dropna()
    m=temp.shape[0]
    l=[]
    for j in range(0,m,2):
        l.append((temp[j],temp[j+1]))
    
    poly = Polygon(l)
    
    NTAmap.loc[c, 'geometry'] = poly
    NTAmap.loc[c,'NTA']= i
    c= c+1

# Uber 2014

In [5]:
date=[]

for i in Uber2014['pickup_datetime']:
    try:
        temp=datetime.datetime.strptime(i, "%m/%d/%y %H:%M")
    except:
        temp=datetime.datetime.strptime(i, "%m/%d/%Y %H:%M:%S")
    
    date.append(temp)

Uber2014['PickDateTime']=date
Uber2014.head()

,pickup_datetime,pickup_latitude,pickup_longitude,base,PickDateTime
0,4/1/14 0:11,40.7690,-73.9549,B02512,2014-04-01 00:11:00
1,4/1/14 0:17,40.7267,-74.0345,B02512,2014-04-01 00:17:00
2,4/1/14 0:21,40.7316,-73.9873,B02512,2014-04-01 00:21:00
3,4/1/14 0:28,40.7588,-73.9776,B02512,2014-04-01 00:28:00
4,4/1/14 0:33,40.7594,-73.9722,B02512,2014-04-01 00:33:00


In [6]:
Uber2014['PickHour']=Uber2014['PickDateTime'].apply(lambda x: x.hour)
Uber2014['Year']=Uber2014['PickDateTime'].apply(lambda x: x.year)
Uber2014['WeekDay']=Uber2014['PickDateTime'].apply(lambda x: x.weekday())
Uber2014['TransportType']='Uber'

In [7]:
Uber2014['PickGeoPoint']=Uber2014.apply(lambda row: Point(row['pickup_longitude'],row['pickup_latitude']),axis=1)
Uber2014PickPoints = gpd.GeoDataFrame()
Uber2014PickPoints['geometry']=Uber2014['PickGeoPoint'].copy()
UMergePoints=gpd.sjoin(Uber2014PickPoints,NTAmap, how="left",op='within')
Uber2014['PickNTA']=UMergePoints['NTA']

In [8]:
Uber2014.head()

,pickup_datetime,pickup_latitude,pickup_longitude,base,PickDateTime,PickHour,Year,WeekDay,TransportType,PickGeoPoint,PickNTA
0,4/1/14 0:11,40.7690,-73.9549,B02512,2014-04-01 00:11:00,0,2014,1,Uber,POINT (-73.95489999999999 40.769),NaN
1,4/1/14 0:17,40.7267,-74.0345,B02512,2014-04-01 00:17:00,0,2014,1,Uber,POINT (-74.03449999999999 40.7267),NaN
2,4/1/14 0:21,40.7316,-73.9873,B02512,2014-04-01 00:21:00,0,2014,1,Uber,POINT (-73.9873 40.7316),MN22
3,4/1/14 0:28,40.7588,-73.9776,B02512,2014-04-01 00:28:00,0,2014,1,Uber,POINT (-73.9776 40.7588),MN17
4,4/1/14 0:33,40.7594,-73.9722,B02512,2014-04-01 00:33:00,0,2014,1,Uber,POINT (-73.9722 40.7594),MN19


## Uber2015

In [9]:
Uber2015['PickDateTime']=pd.to_datetime(Uber2015['pickup_datetime'], format='%Y-%m-%d %H:%M')
Uber2015.tail()

,pickup_datetime,pickup_location_id,dispatch_base,affiliate_base,PickDateTime
14270474,2015-05-08 15:43:00,186,B02765,B02765,2015-05-08 15:43:00
14270475,2015-05-08 15:43:00,263,B02765,B02765,2015-05-08 15:43:00
14270476,2015-05-08 15:43:00,90,B02765,B02765,2015-05-08 15:43:00
14270477,2015-05-08 15:44:00,45,B02765,B01899,2015-05-08 15:44:00
14270478,2015-05-08 15:44:00,144,B02765,B02682,2015-05-08 15:44:00


In [10]:
Uber2015['PickHour']=Uber2015['PickDateTime'].apply(lambda x: x.hour)
Uber2015['Year']=Uber2015['PickDateTime'].apply(lambda x: x.year)
Uber2015['WeekDay']=Uber2015['PickDateTime'].apply(lambda x: x.weekday())
Uber2015['TransportType']='Uber'
Uber2015=pd.merge(Uber2015,Zones[['location_id','nta_code']], how='left', left_on='pickup_location_id',right_on='location_id')
Uber2015.rename(columns={"nta_code": "PickNTA"})

,pickup_datetime,pickup_location_id,dispatch_base,affiliate_base,PickDateTime,PickHour,Year,WeekDay,TransportType,location_id,PickNTA
0,2015-05-17 09:47:00,141,B02617,B02617,2015-05-17 09:47:00,9,2015,6,Uber,141.0,MN31
1,2015-05-17 09:47:00,65,B02617,B02617,2015-05-17 09:47:00,9,2015,6,Uber,65.0,BK38
2,2015-05-17 09:47:00,100,B02617,B02617,2015-05-17 09:47:00,9,2015,6,Uber,100.0,MN13
3,2015-05-17 09:47:00,80,B02617,B02774,2015-05-17 09:47:00,9,2015,6,Uber,80.0,BK90
4,2015-05-17 09:47:00,90,B02617,B02617,2015-05-17 09:47:00,9,2015,6,Uber,90.0,MN13
...,...,...,...,...,...,...,...,...,...,...,...
14270474,2015-05-08 15:43:00,186,B02765,B02765,2015-05-08 15:43:00,15,2015,4,Uber,186.0,MN17
14270475,2015-05-08 15:43:00,263,B02765,B02765,2015-05-08 15:43:00,15,2015,4,Uber,263.0,MN32
14270476,2015-05-08 15:43:00,90,B02765,B02765,2015-05-08 15:43:00,15,2015,4,Uber,90.0,MN13
14270477,2015-05-08 15:44:00,45,B02765,B01899,2015-05-08 15:44:00,15,2015,4,Uber,45.0,MN27


In [11]:
Uber2015.head()

,pickup_datetime,pickup_location_id,dispatch_base,affiliate_base,PickDateTime,PickHour,Year,WeekDay,TransportType,location_id,nta_code
0,2015-05-17 09:47:00,141,B02617,B02617,2015-05-17 09:47:00,9,2015,6,Uber,141.0,MN31
1,2015-05-17 09:47:00,65,B02617,B02617,2015-05-17 09:47:00,9,2015,6,Uber,65.0,BK38
2,2015-05-17 09:47:00,100,B02617,B02617,2015-05-17 09:47:00,9,2015,6,Uber,100.0,MN13
3,2015-05-17 09:47:00,80,B02617,B02774,2015-05-17 09:47:00,9,2015,6,Uber,80.0,BK90
4,2015-05-17 09:47:00,90,B02617,B02617,2015-05-17 09:47:00,9,2015,6,Uber,90.0,MN13


## Green

In [12]:
Green['PickDateTime']=pd.to_datetime(Green['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
Green['DropDateTime']=pd.to_datetime(Green['dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
Green['PickHour']=Green['PickDateTime'].apply(lambda x: x.hour)
Green['Year']=Green['PickDateTime'].apply(lambda x: x.year)
Green['WeekDay']=Green['PickDateTime'].apply(lambda x: x.weekday())

In [13]:
Green['DeltaTime']=Green['DropDateTime']-Green['PickDateTime']


In [ ]:
Green['PickGeoPoint']=Green.apply(lambda row: Point(row['pickup_longitude'],row['pickup_latitude']),axis=1)
GreenPickPoints = gpd.GeoDataFrame()
GreenPickPoints['geometry']=Green['PickGeoPoint'].copy()
GPMergePoints=gpd.sjoin(GreenPickPoints,NTAmap, how="left",op='within')
Green['PickNTA']=GPMergePoints['NTA']



Green['DropGeoPoint']=Green.apply(lambda row: Point(row['dropoff_longitude'],row['dropoff_latitude']),axis=1)
GreenDropPoints = gpd.GeoDataFrame()
GreenDropPoints['geometry']=Green['DropGeoPoint'].copy()
GDMergePoints=gpd.sjoin(GreenDropPoints,NTAmap, how="left",op='within')
Green['DropNTA']=GDMergePoints['NTA']

In [ ]:
Green.head()

## Yellow

In [ ]:
Yellow['PickDateTime']=pd.to_datetime(Yellow['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
Yellow['DropDateTime']=pd.to_datetime(Yellow['dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
Yellow['PickHour']=Yellow['PickDateTime'].apply(lambda x: x.hour)
Yellow['Year']=Yellow['PickDateTime'].apply(lambda x: x.year)
Yellow['WeekDay']=Yellow['PickDateTime'].apply(lambda x: x.weekday())
Yellow['DeltaTime']=Yellow['DropDateTime']-Yellow['PickDateTime']

In [ ]:
Yellow['PickGeoPoint']=Yellow.apply(lambda row: Point(row['pickup_longitude'],row['pickup_latitude']),axis=1)
YellowPickPoints = gpd.GeoDataFrame()
YellowPickPoints['geometry']=Yellow['PickGeoPoint'].copy()
YPMergePoints=gpd.sjoin(YellowPickPoints,NTAmap, how="left",op='within')
Yellow['PickNTA']=YPMergePoints['NTA']



Yellow['DropGeoPoint']=Yellow.apply(lambda row: Point(row['dropoff_longitude'],row['dropoff_latitude']),axis=1)
YellowDropPoints = gpd.GeoDataFrame()
YellowDropPoints['geometry']=Yellow['DropGeoPoint'].copy()
YDMergePoints=gpd.sjoin(YellowDropPoints,NTAmap, how="left",op='within')
Yellow['DropNTA']=YDMergePoints['NTA']

In [ ]:
Yellow.head()

## New York NTA MAP

In [ ]:
sfont=20
NTAmap.plot(alpha=0.5, figsize=(15,11.27), edgecolor='k')
plt.title("New York NTA Map",fontsize=sfont)

plt.xlabel("Longitude [°]",fontsize=sfont)
plt.ylabel("Latitude [°]",fontsize=sfont)